In [70]:
import csv
import requests
import datetime
#!conda install -y bs4
!pip install bs4
from bs4 import BeautifulSoup
import pandas as pd
# the below gives pandas the ability to grab a html table
#!conda install -y html5lib

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [71]:
#def get_dist_ids():
#   """Returns the district IDs from the site."""
#   dist_url = 'http://aahar.jharkhand.gov.in/transactions/transactionView'
#   rids = requests.get(dist_url)
#  content = rids.text
#   soup = BeautifulSoup(content, "html.parser")
#  dist_ids = [(option['value'],option.text) for option in soup.find_all('option', value=True)[1:-2]]
#   return dist_ids

In [72]:
print(get_dist_ids())

[(u'355', u'BOKARO'), (u'347', u'CHATRA'), (u'350', u'DEOGHAR'), (u'354', u'DHANBAD'), (u'362', u'DUMKA'), (u'357', u'EAST SINGHBUM'), (u'346', u'GARHWA'), (u'349', u'GIRIDIH'), (u'351', u'GODDA'), (u'366', u'GUMLA'), (u'360', u'HAZARIBAGH'), (u'363', u'JAMTARA'), (u'348', u'KODERMA'), (u'359', u'LATEHAR'), (u'356', u'LOHARDAGA'), (u'353', u'PAKUR'), (u'358', u'PALAMU'), (u'364', u'RANCHI'), (u'352', u'SAHEBGANJ'), (u'369', u'SARAIKELA KHARSAWAN'), (u'367', u'SIMDEGA'), (u'368', u'WEST SINGHBHUM'), (u'365', u'KHUNTI'), (u'361', u'RAMGARH')]


In [73]:
def get_block_ids(d_id):
    """Returns a list of block ids."""
    block_muni = 'http://aahar.jharkhand.gov.in/dealer_user_logs/getBlockMunicipality'
    dist_post = {'data[DealerUserLog][district_id]':d_id}
    r = requests.post(block_muni, data=dist_post)
    content = r.text
    soup = BeautifulSoup(content, "html.parser")
    block_ids = [(option['value'],option.text) for option in soup.find_all('option', value=True)[1:]]
    #print(block_ids)
    return block_ids

In [74]:
#Want data from 15-8-2016 to 30-11-2016
def get_dates():
    numdays = 109
    base = datetime.date.today()
    date_list = [(base - datetime.timedelta(days=x)).strftime("%d-%m-%Y") for x in range(0, numdays)]
    return(date_list)

In [75]:
def get_dealer_links(block_post):
    r = requests.post("http://aahar.jharkhand.gov.in/dealer_user_logs/hhdLoginDetail", data=block_post)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    main_table = soup.find('table', id='maintable')
    #main_table.find_all('a')
    tmp_link_list = main_table.find_all('a', href=True)
    return(tmp_link_list)
    


In [76]:
def get_transactions(dealer_link_list):
    """Returns transactions."""




In [77]:
#payload = {'data[DealerUserLog][district_id]':'1', 'data[DealerUserLog][block_city_id]':'1',
 #          'data[DealerUserLog][stockdatefrom]': '17-10-2016'}

In [78]:
#r = requests.post("http://aahar.jharkhand.gov.in/dealer_user_logs/hhdLoginDetail", data=payload)
#data = r.text

In [ ]:
#data

In [ ]:
#List for dealer links
dealer_links_list = []
dist_ids = list(range(1,25))
#for d_id in dist_ids:
for d_id in dist_ids:
    b_ids = get_block_ids(d_id)
    print(b_ids)
    for key, value in b_ids:
        date_list = get_dates()[0:10]
        for dt in date_list: 
            block_post = {'data[DealerUserLog][district_id]':key,
                      'data[DealerUserLog][block_city_id]': value,
                      'data[DealerUserLog][stockdatefrom]': dt }
            dealer_links_list.append(get_dealer_links(block_post))
            #Append these links to list
    #list of links
    #need a function that makes requests and processes responses, then calls function to build csv 
            
print(dealer_links_list)       
    

[(u'1', u'BERMO'), (u'420', u'Bokaro Steel City(TA)'), (u'2', u'CHANDANKIYARI'), (u'3', u'CHANDRAPURA'), (u'4', u'CHAS'), (u'258', u'CHAS NAGAR PALIKA(MN)'), (u'5', u'GOMIA'), (u'6', u'JARIDIH'), (u'7', u'KASMAR'), (u'8', u'NAWADIH'), (u'9', u'PETERWAR'), (u'268', u'Phusro (NA)')]
